# Analysis of `vehicle-zone-tracking.xml`

In [19]:
import pprint, math, datetime
from time import strftime, gmtime
from lxml import etree
import matplotlib.pyplot as plt
import numpy as np

from paths import normal, scenarios

plt.rcParams["figure.figsize"] = 30, 20
plt.rcParams["font.size"] = 20
plt.rcParams["axes.titlesize"] = 50
plt.rcParams["axes.titlepad"] = 80

def get_distances_per_zone(file_path, freeze_zones=False):
    class SAXParser:
        def __init__(self):
            self.current_zone_timestep = ""
            self.current_vehicle = {}
            
            self.distances_per_zone = {}

        def start(self, tag, attr):
            if tag == "timestep":
                self.current_zone_timestep = attr["zone-timestep"]
            
            if tag == "vehicle":
                self.current_vehicle = attr

                vid = self.current_vehicle["id"]
                if vid not in self.distances_per_zone:
                    self.distances_per_zone[vid] = {}

            if tag == "polygon":
                distances = self.distances_per_zone[self.current_vehicle["id"]]
                v_timestep = self.current_vehicle["zone-timestep"]
                
                pid, p_timestep = attr["id"].split("_")

                if freeze_zones:
                    if p_timestep != v_timestep:
                        return
                else:
                    if p_timestep != self.current_zone_timestep:
                        return

                if pid.startswith("hole"):
                    hole, hole_counter, zone, polygon_counter = pid.split("-")
                    if zone in distances:
                        distances[zone] -= float(self.current_vehicle["speed"])
                    else:
                        distances[zone] = -float(self.current_vehicle["speed"])
                else:
                    zone, polygon_counter = pid.split("-")
                    if zone in distances:
                        distances[zone] += float(self.current_vehicle["speed"])
                    else:
                        distances[zone] = float(self.current_vehicle["speed"])
                
                    

        def end(self, tag):
            pass

        def close(self):
            pass

    sax_parser = SAXParser()
    xml_parser = etree.XMLParser(target=sax_parser)
    etree.parse(file_path, parser=xml_parser)

    return sax_parser.distances_per_zone

n_distances = get_distances_per_zone(f"{normal}/vehicle-zone-tracking.xml")

scenario_data = {}

for s in scenarios:
    distances = get_distances_per_zone(f"{s}/vehicle-zone-tracking.xml")
    scenario_data[s] = distances

### Distances per trip

In [20]:
pprint.pprint(n_distances)

{'399267601_1': {'01': 890.6585702078103, '02': 408.81091725874575}}


### Distances per person

In [25]:
n_distances_pp = {}

for trip in n_distances:
    person, counter = trip.split("_")

    if person in n_distances_pp:
        person_distances = n_distances_pp[person]
        trip_distances = n_distances[trip]

        for zone in trip_distances:
            if zone in person_distances:
                person_distances[zone] += trip_distances[zone]
            else:
                person_distances[zone] = trip_distances[zone]
                
        n_distances_pp[person] = person_distances

    else:
        n_distances_pp[person] = n_distances[trip]

pprint.pprint(n_distances_pp)


{'399267601': {'01': 890.6585702078103, '02': 408.81091725874575}}
